
# Data preparation and Cleaning

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
# Load the tables
sentiment = pd.read_csv('sentiment_statistics.csv')
customers = pd.read_csv('customers.csv')
reason = pd.read_csv('reason.csv')
calls=pd.read_csv('/content/calls.csv')

In [6]:
# Describe sentiment_statistic table
print(sentiment.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71810 entries, 0 to 71809
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   call_id                  71810 non-null  int64  
 1   agent_id                 71810 non-null  int64  
 2   agent_tone               71593 non-null  object 
 3   customer_tone            71810 non-null  object 
 4   average_sentiment        71701 non-null  float64
 5   silence_percent_average  71810 non-null  float64
dtypes: float64(2), int64(2), object(2)
memory usage: 3.3+ MB
None


In [7]:
# Describe customers table
print(customers.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71810 entries, 0 to 71809
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       71810 non-null  int64  
 1   customer_name     71810 non-null  object 
 2   elite_level_code  46043 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.6+ MB
None


In [8]:
#Describe reason_table
print(reason.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66653 entries, 0 to 66652
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   call_id              66653 non-null  int64 
 1   primary_call_reason  66653 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.0+ MB
None


In [9]:
print(calls.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71810 entries, 0 to 71809
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   call_id                  71810 non-null  int64 
 1   customer_id              71810 non-null  int64 
 2   agent_id                 71810 non-null  int64 
 3   call_start_datetime      71810 non-null  object
 4   agent_assigned_datetime  71810 non-null  object
 5   call_end_datetime        71810 non-null  object
 6   call_transcript          71810 non-null  object
dtypes: int64(3), object(4)
memory usage: 3.8+ MB
None


Handling Missing values

In [10]:
#Percentage of missing values in each column of Sentiment_statistic table
total = sentiment.isnull().sum().sort_values(ascending = False)
percent = (sentiment.isnull().sum()/sentiment.isnull().count()).sort_values(ascending = False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(10)

,Total,Percent
agent_tone,217,0.003022
average_sentiment,109,0.001518
call_id,0,0.000000
agent_id,0,0.000000
customer_tone,0,0.000000
silence_percent_average,0,0.000000


In [11]:
#Removing Outliers
# Create a boxplot for average_sentiment
plt.figure(figsize=(8, 5))
sns.boxplot(sentiment['average_sentiment'])
plt.title('Boxplot of Average Sentiment')
plt.ylabel('Average Sentiment')
plt.show()

NameError: name 'sns' is not defined

<Figure size 800x500 with 0 Axes>

In [12]:
# Calculate the lower and upper thresholds
Q1 = sentiment['average_sentiment'].quantile(0.25)
Q3 = sentiment['average_sentiment'].quantile(0.75)
IQR = Q3 - Q1
lower_threshold = Q1 - 1.5 * IQR
upper_threshold = Q3 + 1.5 * IQR

# Replace outliers with threshold values
sentiment['average_sentiment'] = sentiment['average_sentiment'].apply(
    lambda x: lower_threshold if x < lower_threshold else (upper_threshold if x > upper_threshold else x)
)


In [13]:
skewness = sentiment['average_sentiment'].skew()
print(skewness)

-0.1533594430972926


Used mean to replace missing values because a skewness value of -0.153 indicates that the distribution of
average_sentiment is slightly negatively skewed, but it is very close to zero. This suggests that the
distribution is approximately symmetrical and not significantly skewed.

In [14]:
#Replacing missing values
sentiment['agent_tone'] = sentiment['agent_tone'].fillna(sentiment['agent_tone'].mode()[0])
sentiment['average_sentiment'] = sentiment['average_sentiment'].fillna(sentiment['average_sentiment'].mean())


In [15]:
# Percentage of missing values in each column of Customers table
total = customers.isnull().sum().sort_values(ascending = False)
percent = (customers.isnull().sum()/customers.isnull().count()).sort_values(ascending = False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(10)

,Total,Percent
elite_level_code,25767,0.358822
customer_id,0,0.000000
customer_name,0,0.000000


In [16]:
# Replace missing values in elite_level_code of customers
customers['elite_level_code']=customers['elite_level_code'].fillna('NaN')

# Data Tranformation

In [17]:
# Rename the column of customers table
customers.rename(columns={'elite_level_code': 'mp_status'}, inplace=True)
#customers.head(10)

In [18]:
# Tranforming columns of calls table
# Convert datetime columns to proper datetime format

calls['call_start_datetime'] = pd.to_datetime(calls['call_start_datetime'], format='%m/%d/%Y %H:%M')
calls['agent_assigned_datetime'] = pd.to_datetime(calls['agent_assigned_datetime'], format='%m/%d/%Y %H:%M')
calls['call_end_datetime'] = pd.to_datetime(calls['call_end_datetime'], format='%m/%d/%Y %H:%M')

# Step 1: Separate date and time
calls['call_start_date'] = calls['call_start_datetime'].dt.date
calls['call_start_time'] = calls['call_start_datetime'].dt.time

calls['agent_assigned_date'] = calls['agent_assigned_datetime'].dt.date
calls['agent_assigned_time'] = calls['agent_assigned_datetime'].dt.time

calls['call_end_date'] = calls['call_end_datetime'].dt.date
calls['call_end_time'] = calls['call_end_datetime'].dt.time

# Format date as dd mm yyyy
calls['call_start_date'] = calls['call_start_date'].apply(lambda x: x.strftime('%d %m %Y'))
calls['agent_assigned_date'] = calls['agent_assigned_date'].apply(lambda x: x.strftime('%d %m %Y'))
calls['call_end_date'] = calls['call_end_date'].apply(lambda x: x.strftime('%d %m %Y'))

# Step 2: Calculate call duration and agent response time
calls['call_duration'] = (calls['call_end_datetime'] - calls['call_start_datetime']).dt.total_seconds() / 60  # Duration in minutescalls['agent_response_time'] = calls['agent_assigned_datetime'] - calls['call_start_datetime']
calls['agent_response_time'] = (calls['agent_assigned_datetime'] - calls['call_start_datetime']).dt.total_seconds() / 60

# Step 3: Extract time features
calls['call_start_day'] = calls['call_start_datetime'].dt.dayofweek  # 0 = Monday, 6 = Sunday


In [19]:
# Creating master dataset
# Merge tables on call_id
merged_df = calls.merge(reason, on='call_id', how='left')
merged_df = merged_df.merge(sentiment, on=['call_id', 'agent_id'], how='left')
merged_df = merged_df.merge(customers, on='customer_id', how='left')

# Inspect the final merged table
merged_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71810 entries, 0 to 71809
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   call_id                  71810 non-null  int64         
 1   customer_id              71810 non-null  int64         
 2   agent_id                 71810 non-null  int64         
 3   call_start_datetime      71810 non-null  datetime64[ns]
 4   agent_assigned_datetime  71810 non-null  datetime64[ns]
 5   call_end_datetime        71810 non-null  datetime64[ns]
 6   call_transcript          71810 non-null  object        
 7   call_start_date          71810 non-null  object        
 8   call_start_time          71810 non-null  object        
 9   agent_assigned_date      71810 non-null  object        
 10  agent_assigned_time      71810 non-null  object        
 11  call_end_date            71810 non-null  object        
 12  call_end_time            71810 n

In [20]:
merged_df.to_csv('master_table.csv', index=False) # Master table after combining all the datasets

In [21]:
merged_df.head()

,call_id,customer_id,agent_id,call_start_datetime,agent_assigned_datetime,call_end_datetime,call_transcript,call_start_date,call_start_time,agent_assigned_date,...,call_duration,agent_response_time,call_start_day,primary_call_reason,agent_tone,customer_tone,average_sentiment,silence_percent_average,customer_name,mp_status
0,4667960400,2033123310,963118,2024-07-31 23:56:00,2024-08-01 00:03:00,2024-08-01 00:34:00,\n\nAgent: Thank you for calling United Airlin...,31 07 2024,23:56:00,01 08 2024,...,38.0,7.0,2,Voluntary Cancel,neutral,angry,-0.04,0.39,Matthew Foster,4.0
1,1122072124,8186702651,519057,2024-08-01 00:03:00,2024-08-01 00:06:00,2024-08-01 00:18:00,\n\nAgent: Thank you for calling United Airlin...,01 08 2024,00:03:00,01 08 2024,...,15.0,3.0,3,Booking,calm,neutral,0.02,0.35,Tammy Walters,NaN
2,6834291559,2416856629,158319,2024-07-31 23:59:00,2024-08-01 00:07:00,2024-08-01 00:26:00,\n\nAgent: Thank you for calling United Airlin...,31 07 2024,23:59:00,01 08 2024,...,27.0,8.0,2,IRROPS,neutral,polite,-0.13,0.32,Jeffery Dixon,NaN
3,2266439882,1154544516,488324,2024-08-01 00:05:00,2024-08-01 00:10:00,2024-08-01 00:17:00,\n\nAgent: Thank you for calling United Airlin...,01 08 2024,00:05:00,01 08 2024,...,12.0,5.0,3,Upgrade,neutral,frustrated,-0.20,0.20,David Wilkins,2.0
4,1211603231,5214456437,721730,2024-08-01 00:04:00,2024-08-01 00:14:00,2024-08-01 00:23:00,\n\nAgent: Thank you for calling United Airlin...,01 08 2024,00:04:00,01 08 2024,...,19.0,10.0,3,Seating,neutral,polite,-0.05,0.35,Elizabeth Daniels,0.0
